# Nonlinear mixed-effects models in Julia

## Linear mixed-effects models

A linear mixed-effects model is characterized by the distribution of
two vector-valued random variables: the $n$-dimensional response, $Y$,
and the $q$-dimensional random effects vector, $B$.  The unconditional
distribution of $B$ is multivariate normal
$$
    B\sim N(0,\Sigma_\theta)
$$
as is the conditional distribution of $Y$ given $B=b$
$$
    (Y|B=b)\sim N(X\beta+Zb, \sigma^2 I)
$$

In the [MixedModels](https://github.com/dmbates/MixedModels) package for 
[Julia](http://julialang.org) we represent the covariance matrix in the unconditional distribution of $B$ as
$$
    \Sigma_\theta=\sigma^2\Lambda_\theta\Lambda_\theta'
$$
where $\Lambda_\theta$ is the $q\times q$ _relative covariance factor_.

For given values of $\theta$ and $\beta$ we solve a penalized linear least squares problem of the form
$$
    r^2_{\beta,\theta}=\min_u \|(y-X\beta)-Z\Lambda_\theta u\|^2 + \|u\|^2
$$
for which we compute the sparse Cholesky factorization
$$
    L_\theta L_\theta' = \Lambda_\theta'Z'Z\Lambda_\theta + I
$$
where $L_\theta$ is a lower-triangular sparse matrix.
Because $L_\theta$ is triangular, we can easily evaluate its determinant as the product of its diagonal elements.
By construction these diagonal elements are positive and the log-determinant, $\log(|\Lambda_\theta'Z'Z\Lambda_\theta + I|)=2\log(|L_\theta|)$ is easily evaluated.

The log-likelihood for a linear mixed-effects model, $\ell(\beta,\theta|y)$, can be expressed as
$$
    -2\ell(\beta,\theta|y) = \log(|\Lambda_\theta'Z'Z\Lambda_\theta + I|)+n\left[1+\log\left(\frac{2\pi r^2_{\beta,\theta}}{n}\right)\right]
$$

## Nonlinear mixed-effects models

The formulation of a nonlinear mixed-effects model (NLMM) is nearly the same as that of an LMM except that the mean of the conditional distribution $Y|B=b$ is a nonlinear function of the parameters $\beta$ and the random effects, $b$ or, equivalently, the _spherical_ random effects $u$ where $b=\Lambda_\theta u$.

The nonlinear model function, $f$, is a function of a $k$-dimensional model parameter, $\phi$ and covariates. In our formulation we write the `vec` of the $n\times k$ _model parameter matrix_, $\Phi$ as a linear predictor
$$
    vec(\Phi)=X\beta+Z\Lambda_\theta u
$$
and evaluate
$$
    \mu_{Y|U=u}=f(T,\Phi)
$$
where $T$ is a matrix of covariate values, _time_ in the case of a population pharmacokinetic model.

The penalized linear least squares (PLS) problem solved for each evaluation of the objective in the case of a linear mixed-effects model is replaced by a penalized nonlinear least squares (PNLS) problem
$$
    r^2_{\beta,\theta}=\min_u \|y-\mu_{Y|U=u}\|^2+\|u\|^2
$$
in an NLMM.  
The optimizer, $\tilde{u}=\arg\min_u r^2_{\beta,\theta}$, is the _mode_ of the distribution of $U$ given $Y=y$ and the current values of the parameters, $\beta$ and $\theta$.

In general there will not be a closed-form expression for the log-likelihood.  However, the _Laplace approximation_ to the log-likelihood is exactly the same expression as the log-likelihood for an LMM. The Laplace approximation is equivalent to a one-point adaptive Gauss-Hermite approximation to the integral that defines the log-likelihood.  This approximation is _adaptive_ in the sense that the Gauss-Hermite approximation is taken at the mode of the conditional distribution, not at the mean of the unconditional distribution.

## A simple example - first-order kinetics, 1 compartment, single bolus unit dose

One of the simplest pharmacokinetics models is that for a one compartment model with a single bolus dose.
The predicted concentration at time $t$ is given by
$$
        c(t)=V\exp(-Kt)
$$
where V is the volume of distribution and K is the elimination rate constant.

## Fitting the model

Simulated data from a one compartment model with a single bolus dose are provided in a compressed `csv` file in the `data` directory of the `NLreg` package.  We read these data as a `DataFrame` object.

In [2]:
using CSV, DataFrames, LinearAlgebra, NLreg
const datadir = normpath(joinpath(dirname(pathof(NLreg)), "..", "data"));

In [3]:
sd1 = CSV.read(joinpath(datadir, "sd1.csv"));
describe(sd1)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,id,,S001,,S200,200,,String
2,time,3.89914,0.5,4.0,9.0,,,Float64
3,conc,0.541184,0.0012369,0.35909,3.4862,,,Float64


To fit all the data, without taking the different subjects into account, with parameters representing `log(V)` and `log(K)`, use a `NLregModel`.
The first argument is a function of parameters, `p`, and a data table (in the sense of the [`Tables` package](https://github.com/JuliaData/Tables.jl) ), `d`, that evaluates the mean response vector.  Usually the expression to evaluate the response is a call to the `@.` macro for implicit broadcasting.  The [`ForwardDiff` package](https://github.com/JuliaDiff/ForwardDiff.jl) is used with this function to evaluate the Jacobian matrix along with the mean response.  Subsequent arguments are the data, starting values for the parameters, as a `NamedTuple`, and a symbol giving the name of the response column in the data table.

In [4]:
mfixed = fit!(NLregModel((p,d) -> @.(exp(p[1] - exp(p[2])*d.time)), sd1, (lV = -1.0, lK = -1.0), :conc), verbose=true)

(cvg, oldrss, φ) = (0.7185936303492759, 229.6957043513567, [-1.0, -1.0])
(cvg, oldrss, φ) = (0.12007347686490066, 112.30911051106428, [0.021532292080037818, -1.6643720635642465])
(cvg, oldrss, φ) = (0.011649172168248697, 110.61229713814498, [0.12519084059271413, -1.4041640998848468])
(cvg, oldrss, φ) = (4.984323753092575e-5, 110.59728661059417, [0.13364076029214383, -1.4037274383268128])
(cvg, oldrss, φ) = (7.782713197783087e-7, 110.59728633478112, [0.13361607107919604, -1.403703244658451])


Nonlinear regression model (NLregModel)

 Residual sum of squares: 110.59728633478112

─────────────────────────────────
     Estimate  Std.Error  t value
─────────────────────────────────
lV   0.133616  0.043366      3.08
lK  -1.4037    0.0823868   -17.04
─────────────────────────────────


The parameter estimates from this fit provide starting estimates for the fixed-effects in a nonlinear mixed-effects model

In [5]:
mmixed = NLmixedModel((p,d) -> @.(exp(p[1] - exp(p[2])*d.time)), groupby(sd1, :id), :conc, params(mfixed));

The relative covariance factor, $\lambda$, has been initialized to the identity.

In [6]:
mmixed.λ

2×2 LowerTriangular{Float64,Array{Float64,2}}:
 1.0   ⋅ 
 0.0  1.0

corresponding to a $\theta$ parameter of

In [7]:
mmixed.θ

3-element Array{Float64,1}:
 1.0
 0.0
 1.0

The _penalized nonlinear least squares_ (pnls) algorithm minimizes $\|y-\eta(\varphi,\mathbf{u})\|^2 + \|\mathbf{u}\|^2$ with respect to $\varphi$ and $\mathbf{u}$

In [8]:
pnls!(mmixed, verbose=true);

(cvg, oldprss, φ) = (0.9464185875221391, 110.59728633478106, [0.13361607107919604, -1.403703244658451])
(cvg, oldprss, φ) = (0.06269475212675366, 49.76367117639907, [0.1369620839731903, -1.3992244104924974])
(cvg, oldprss, φ) = (0.0017953893023616664, 46.88563589804223, [0.20151422964846882, -1.2907798896696423])
(cvg, oldprss, φ) = (0.00010908198566120474, 46.809751770025066, [0.19641569400621764, -1.2921697813893755])
(cvg, oldprss, φ) = (1.2708227190782454e-5, 46.805894799807554, [0.2014658183448791, -1.2845690897110407])
(cvg, oldprss, φ) = (1.8019353728133105e-6, 46.80549294111189, [0.20077963436467372, -1.28505476179701])
(cvg, oldprss, φ) = (2.89456177375038e-7, 46.80544004206064, [0.201254067082151, -1.2843255022306415])
(cvg, oldprss, φ) = (4.9315424647577386e-8, 46.80543190558488, [0.2011566070226851, -1.284438098063897])
(cvg, oldprss, φ) = (8.733079296364763e-9, 46.80543055401347, [0.201209801488745, -1.2843517998025566])
(cvg, oldprss, φ) = (1.5739730522205883e-9, 46.80543

At the minimum, the Laplace approximation to negative twice the log-likelihood is

In [9]:
objective(mmixed)

394.6577176584259

and the conditional modes of the spherical random effects are

In [10]:
mmixed.u

2×200 Array{Float64,2}:
 -0.293802   -0.101465   -0.160315   …   0.0643369  -0.318807  -0.119347 
  0.0977617   0.0282295   0.0670755     -0.131454    0.175484   0.0915935

The objective is evaluated just as for the linear mixed-effects model
$$
    -2\ell(\theta|y) = \log(|\Lambda_\theta'Z'Z\Lambda_\theta + I|)+n\left[1+\log\left(\frac{2\pi r^2_{\theta}}{n}\right)\right]
$$
where the first term, the logarithm of the determinant, whose value is

In [11]:
logdet(mmixed)

208.5657672146824

is evaluated from the Cholesky factor of $\Lambda_\theta'Z'Z\Lambda_\theta + I$.
This is a block-diagonal positive-definite symmetric matrix consisting of 200 diagonal blocks of size $2\times2$.
The lower Cholesky factors of each block are in the `L11` field of the `NLmixedModel` object.

In [12]:
typeof(mmixed.L11)

Array{Array{Float64,2},1}

In [13]:
LowerTriangular(first(mmixed.L11))

2×2 LowerTriangular{Float64,Array{Float64,2}}:
  1.31741    ⋅     
 -0.263603  1.07487

Now all that is needed is to optimize the objective with respect to $\theta$ subject to the constraint that the elements of $\theta$ corresponding to diagonal
elements of $\lambda$ are non-negative.

In [14]:
using NLopt
opt = NLopt.Opt(:LN_BOBYQA, 3)
min_objective!(opt, (x, g) -> objective(pnls!(NLreg.setθ!(mmixed, x))))
lower_bounds!(opt, [0.0, -Inf, 0.0])
optf, optx, ret = optimize(opt, [1.0, 0.0, 1.0])

(185.79025122840824, [3.5223248474429947, -0.22893688140116186, 2.836161273379503], :ROUNDOFF_LIMITED)

The estimate of the fixed-effects parameter is

In [15]:
mmixed.φ

2-element Array{Float64,1}:
  0.079226478961321 
 -1.2731020331580518

The conditional mode of the "spherical" random effects is

In [16]:
mmixed.u

2×200 Array{Float64,2}:
 -0.163124   -0.051526   -0.0529521  …   0.0462964  -0.16779    -0.0464034
 -0.0140483  -0.0289852   0.0380786     -0.122477    0.0962269   0.060299 

or, on the original scale of `log(V)` and `log(K)`

In [17]:
mmixed.λ * mmixed.u

2×200 Array{Float64,2}:
 -0.574575    -0.181491   -0.186515  …   0.163071  -0.59101   -0.163448
 -0.00249822  -0.0704106   0.12012      -0.357965   0.311328   0.181641

The estimate, $\hat{\sigma}$ of the standard deviation of the per-observation error is

In [18]:
dispersion(mmixed)

0.16089136749363414

which is the penalized residual sum of squares divided by the number of observations.

To evaluate the standard deviations of the within-subject random effects, it helps first to evaluate the row lengths of $\lambda$

In [19]:
rowlengths = [norm(view(mmixed.λ, i, 1:i)) for i in 1:size(mmixed.λ, 1)]

2-element Array{Float64,1}:
 3.5223248474429947
 2.8453862416697198

From this vector the estimated standard deviations of the random effects are evaluated as

In [20]:
dispersion(mmixed) .* rowlengths

2-element Array{Float64,1}:
 0.5667116614619097
 0.4577980834698134

and the within-subject correlation matrix as

In [21]:
rhofac = ldiv!(Diagonal(rowlengths), copy(mmixed.λ));
rhofac * rhofac'

2×2 Array{Float64,2}:
  1.0       -0.080459
 -0.080459   1.0     

## Examples from chapter 8 of Pinheiro and Bates (Springer, 2000)

### Orange Tree Growth

In [22]:
Orange = CSV.read(joinpath(datadir, "Orange.csv"));
describe(Orange)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,tree,,T1,,T5,5,,String
2,age,922.143,118.0,1004.0,1582.0,,,Float64
3,circumference,115.857,30.0,115.0,214.0,,,Float64


Define the logistic growth model in terms of three parameters, `Asym`, the upper asymptote, `xmid`, the `x` value at which the predicted response is `Asym/2`, and `scal`, the horizontal scale factor.  See appendix C.7 and Figure C.7, p. 519.

In [23]:
logisgrowth(p, d) = @.(p[1]/(1+exp(-(d.age - p[2]) / p[3])));

In [24]:
ofixed = fit!(NLregModel(logisgrowth, Orange, (Asym = 200., xmid = 700., scal = 350.), :circumference), verbose=true)

(cvg, oldrss, φ) = (0.3180264427223918, 19447.214945911193, [200.0, 700.0, 350.0])
(cvg, oldrss, φ) = (0.005131657142795473, 17480.694137576265, [192.55512163222824, 726.9341243980331, 353.33466236771443])
(cvg, oldrss, φ) = (5.706203420504341e-5, 17480.233568919328, [192.68126148038633, 728.7275476881084, 353.5063185097269])
(cvg, oldrss, φ) = (2.8865929273658546e-6, 17480.23350919397, [192.68726772555488, 728.7547583488711, 353.53228967473086])


Nonlinear regression model (NLregModel)

 Residual sum of squares: 17480.23350919397

──────────────────────────────────
      Estimate  Std.Error  t value
──────────────────────────────────
Asym   192.687    20.2439     9.52
xmid   728.755   107.297      6.79
scal   353.532    81.4714     4.34
──────────────────────────────────


In [25]:
omixed = NLmixedModel(logisgrowth, groupby(Orange, :tree), :circumference, params(ofixed));
objective(pnls!(omixed, verbose=true))

(cvg, oldprss, φ) = (2.1110236844753305, 17480.23350919397, [192.68726772555488, 728.7547583488711, 353.53228967473086])
(cvg, oldprss, φ) = (0.027285112273739223, 5617.006499257276, [192.68757523908965, 728.7562998347742, 353.5335943247664])
(cvg, oldprss, φ) = (0.003310385482357307, 5505.013392969411, [182.9684635235851, 679.0250970245471, 313.89186266975423])
(cvg, oldprss, φ) = (9.041043807996186e-5, 5489.554604018819, [185.41825878680675, 691.8587989437445, 322.5103952153069])
(cvg, oldprss, φ) = (3.619144328782728e-6, 5489.151043789456, [185.11314444262763, 690.2459081644697, 320.7836863383409])
(cvg, oldprss, φ) = (1.4442929685751778e-7, 5489.135129541503, [185.19165480944366, 690.6636715506445, 321.1397248434211])
(cvg, oldprss, φ) = (5.874718537739253e-9, 5489.1344964229875, [185.17691930964048, 690.5865637437571, 321.0696081178872])
(cvg, oldprss, φ) = (2.3849695176077213e-10, 5489.134470672978, [185.17997656793875, 690.6026546121702, 321.0838953741613])


283.9684679762638

In [26]:
opt = NLopt.Opt(:LN_BOBYQA, 6)
obj(x, g) = objective(pnls!(NLreg.setθ!(omixed, x)))
min_objective!(opt, obj)
lower_bounds!(opt, [0.0, -Inf, -Inf, 0.0, -Inf, 0.0])
optf, optx, ret = optimize(opt, omixed.θ)

(259.9416188441591, [3.4315183328653402, -2.71632364140438, -5.7950218864731315, 6.24985021606303e-5, 9.717036816618956e-6, 3.883325865577966e-7], :ROUNDOFF_LIMITED)

Notice that the estimated covariance of the random-effects is essentially singular.

In [27]:
omixed.λ

3×3 LowerTriangular{Float64,Array{Float64,2}}:
  3.43152   ⋅           ⋅        
 -2.71632  6.24985e-5   ⋅        
 -5.79502  9.71704e-6  3.88333e-7

In, fact the covariance matrix for the random effect is more-or-less a rank 1 matrix, instead of rank 3.
This should not be surprising because there is only date from 5 trees.
Expecting to estimate a total of 7 variance-covariance parameters and 3 fixed-effects parameters from data on only 5 trees is optimistic.

For the record, the estimates of the fixed-effects parameters are

In [28]:
omixed.φ

3-element Array{Float64,1}:
 193.24012197656984
 735.3113198833935 
 362.12237344029967

and the estimate of $\sigma$ is

In [29]:
dispersion(omixed)

7.428378899804974

The modes of the spherical random effects are

In [30]:
omixed.u

3×5 Array{Float64,2}:
 -6.97096      7.36877     -8.83959      9.33449     -0.8927     
 -5.39391e-5  -7.68977e-5   8.75173e-6  -5.60133e-6   0.000127686
  4.6132e-7    3.89683e-7   7.88651e-8  -2.22545e-7  -7.07323e-7 

or, on the original scale of `Asym`, `xmid`, and `scal`

In [31]:
omixed.λ * omixed.u

3×5 Array{Float64,2}:
 -23.921    25.2861  -30.3332   32.0315  -3.06332
  18.9354  -20.016    24.0112  -25.3555   2.42486
  40.3969  -42.7022   51.2256  -54.0936   5.17322

## Theophylline

In [32]:
Theo = CSV.read(joinpath(datadir, "Theophylline.csv"));
describe(Theo)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Subj,,S01,,S12,12,,String
2,Wt,69.5833,54.6,70.5,86.4,,,Float64
3,dose,4.62583,3.1,4.53,5.86,,,Float64
4,time,5.89462,0.0,3.53,24.65,,,Float64
5,conc,4.96045,0.0,5.275,11.4,,,Float64


Each subject received a single, oral dose at time 0.  A one-compartment model for these data can be expressed in terms of `lk`, the log of the elimination rate constant, `lka`, the log of the absorption rate constant, and `lV`, the log of the effective volume of distribution as

In [33]:
function sdOral1C(φ, data)
    k  = exp(φ[1])    # elimination rate constant from lk
    ka = exp(φ[2])    # absorption rate constant from lka
    V  = exp(φ[3])    # volume of distribution from lV
    t  = data.time
    @. (data.dose/V) * (ka/(ka - k)) * (exp(-k*t) - exp(-ka*t))
end

sdOral1C (generic function with 1 method)

In [34]:
tfixed = fit!(NLregModel(sdOral1C, Theo, (lk = -2.5, lka = 0.5, lV = -1.0), :conc), verbose=true)

(cvg, oldrss, φ) = (0.7853356271545054, 715.6711644323042, [-2.5, 0.5, -1.0])
(cvg, oldrss, φ) = (0.14274745094116054, 280.1524269431178, [-2.5163796453795224, 0.4178478303222412, -0.7597743655417365])
(cvg, oldrss, φ) = (0.0025774759291832977, 274.4509457286508, [-2.5231736672520815, 0.39839314053529373, -0.7250618599859671])
(cvg, oldrss, φ) = (6.49494920608347e-5, 274.449135628282, [-2.52429096531231, 0.39930574924190987, -0.7239983016514161])
(cvg, oldrss, φ) = (5.963695719867695e-6, 274.44913457677944, [-2.52423410870312, 0.3992193504126945, -0.7240261939831345])


Nonlinear regression model (NLregModel)

 Residual sum of squares: 274.44913457677944

──────────────────────────────────
      Estimate  Std.Error  t value
──────────────────────────────────
lk   -2.52423    0.110347   -22.88
lka   0.399219   0.117537     3.40
lV   -0.724026   0.04858    -14.90
──────────────────────────────────


In [38]:
tmixed = NLmixedModel(sdOral1C, groupby(Theo, :Subj), :conc, params(tfixed));
objective(pnls!(tmixed, verbose=true))

(cvg, oldprss, φ) = (3.740164584359218, 274.44913457677956, [-2.52423410870312, 0.3992193504126945, -0.7240261939831345])
(cvg, oldprss, φ) = (0.2538644283762077, 65.45355317646988, [-2.539724608346645, 0.5159034644310972, -0.735849519021081])
(cvg, oldprss, φ) = (0.045903458732935436, 54.586945808304264, [-2.427056127701912, 0.42264106959083503, -0.7797569538875166])
(cvg, oldprss, φ) = (0.006322750981342246, 53.0249011390628, [-2.4436478745301433, 0.4733461541547802, -0.7744955485066873])
(cvg, oldprss, φ) = (0.002582764801759987, 52.864652255324074, [-2.435048274003457, 0.45020283172398856, -0.7794250917666978])
(cvg, oldprss, φ) = (0.0006327381846407471, 52.795970165312205, [-2.4384625732558014, 0.46535316899351764, -0.7775831980923442])
(cvg, oldprss, φ) = (0.00027356312342152636, 52.78429704906995, [-2.43622337962098, 0.4570052863430945, -0.7788038848180524])
(cvg, oldprss, φ) = (8.583842316351474e-5, 52.77795206777611, [-2.437408899420888, 0.46215948798151774, -0.778157426369326

393.57025145060913

In [39]:
obj(x,g) = objective(pnls!(NLreg.setθ!(tmixed, x)))
min_objective!(opt, obj)
optf, optx, ret = optimize(opt, tmixed.θ)

(346.6272637900606, [0.1904026843870456, 0.015428917813319794, 0.17674043831369401, 0.957689294185729, -0.03916097637545407, 0.0], :ROUNDOFF_LIMITED)

The value of the objective, which is on the deviance scale of negative twice the log-likelihood, corresponds to a log-likelihood of

In [40]:
-optf/2

-173.3136318950303

which is essentially the same as that in Pinheiro and Bates (2000), although arrived at by a very different optimization mechanism.

The estimates of the fixed-effects parameters

In [41]:
tmixed.φ

3-element Array{Float64,1}:
 -2.433144119963929  
  0.45265253825801366
 -0.7814700851185019 

match those from the previous analysis.

As for model `omixed` the estimate of the covariance matrix of the random effects, based on

In [42]:
tmixed.λ

3×3 LowerTriangular{Float64,Array{Float64,2}}:
 0.190403     ⋅         ⋅ 
 0.0154289   0.957689   ⋅ 
 0.17674    -0.039161  0.0

will be rank-deficient - in this case it will be rank 2.

In [43]:
rowlengths = [norm(view(tmixed.λ, i, 1:i)) for i in 1:3];
dispersion(tmixed) .* rowlengths  # standard deviations of the random effects

3-element Array{Float64,1}:
 0.12948367386045226
 0.6513627702893523 
 0.1231077045107887 

and the correlation matrix is

In [44]:
ρfac = ldiv!(Diagonal(rowlengths), copy(tmixed.λ))
ρfac * ρfac'

3×3 Array{Float64,2}:
 1.0         0.0161085   0.976321
 0.0161085   1.0        -0.200572
 0.976321   -0.200572    1.0     

Note that one can't tell easily from the values in the correlation matrix that it is rank deficient.